In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfTransformer

time: 330 ms (started: 2022-05-21 08:03:24 +00:00)


# Load data

In [3]:
with open("./datasets/test_leader_session.pkl" , "rb") as f:
    tmp_output = pickle.load(f)
    

candidate_df = pd.read_csv("./datasets/candidate_vector.csv").set_index("item_id")
session_df = pd.DataFrame.from_dict(tmp_output, orient='index')

# print(f"session_df shape:{session_df.shape}")
# display(session_df.head())
# print(f"candidate_df shape:{candidate_df.shape}")
# display(candidate_df.head())

time: 6.71 s (started: 2022-05-21 08:03:42 +00:00)


# Build tfidf vector
- turn `session_df` to tfidf vector
- turn `candidte_df` to tfidf vector

In [4]:
model = TfidfTransformer()
model.fit(session_df)
session_tfidf_sparse_mat = model.transform(session_df)
session_tfidf_arr = session_tfidf_sparse_mat.toarray()
candidate_tfidf_sparse_mat = model.transform(candidate_df)
candidate_tfidf_arr = candidate_tfidf_sparse_mat.toarray()

# print(f"candidate_tfidf_arr shape:{candidate_tfidf_arr.shape}")
# display(candidate_tfidf_arr[:5])
# print(f"session_tfidf_arr shape:{session_tfidf_arr.shape}")
# display(session_tfidf_arr[:5])

time: 667 ms (started: 2022-05-21 08:03:56 +00:00)


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:438: UserWarning: X has feature names, but TfidfTransformer was fitted without feature names
  warnings.warn(


# Compute similarity 
- similarity bewteen session_tfidf and candidate_tfidf
- rank by similarity

In [5]:
sim_arr = np.dot(session_tfidf_arr,candidate_tfidf_arr.T)
candidate_ri_df = candidate_df.reset_index()
argsort_sim_arr = np.argsort(sim_arr)
item_id_rank_arr = np.apply_along_axis(lambda x:candidate_ri_df.loc[x[:100],"item_id"], 1, argsort_sim_arr).reshape(-1)

print(candidate_ri_df.loc[[3898,4000],"item_id"])
display(argsort_sim_arr[:10])
display(item_id_rank_arr[:10])

3898    21955
4000    22576
Name: item_id, dtype: int64


array([[3584, 2654, 3041, ..., 2904,  605, 3403],
       [ 283, 1542, 4447, ..., 3075, 3036,  271],
       [2654, 3041, 3952, ..., 1180, 2288, 1483],
       ...,
       [2654, 2468, 4396, ..., 1204, 2619, 1991],
       [2654,  443, 4275, ..., 4656, 2113, 4137],
       [4682, 3806, 3584, ...,   36, 2176,  553]])

array([20224, 15032, 17106, 22276, 12358,  5691, 11206, 18609, 10458,
        5165])

time: 21.9 s (started: 2022-05-21 08:03:56 +00:00)


In [6]:
# Output results

time: 319 µs (started: 2022-05-21 08:04:18 +00:00)


In [7]:
res_df = pd.DataFrame(np.repeat(session_df.index, 100), columns=["session_id"])
res_df["item_id"] = item_id_rank_arr
res_df["rank"] = (list(range(1,101))*len(session_df))
res_df.to_csv("./datasets/results/test_leaderboard_0521_1.csv", index=False)

observe_ind = np.array([[i*100+j for j in range(10)] for i in range(len(session_df))]).reshape(-1)
res_df.iloc[observe_ind,:][:50]

,session_id,item_id,rank
0,26,20224,1
1,26,15032,2
2,26,17106,3
3,26,22276,4
4,26,12358,5
5,26,5691,6
6,26,11206,7
7,26,18609,8
8,26,10458,9
9,26,5165,10


time: 4.42 s (started: 2022-05-21 08:04:18 +00:00)


In [ ]:
# np.arange(1000)